In [135]:
# import libraries
import boto3, re, sys, math, json, os, sagemaker, urllib.request
from sagemaker import get_execution_role
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image
from IPython.display import display
from time import gmtime, strftime
from sagemaker.predictor import csv_serializer

# Define IAM role
role = get_execution_role()
prefix = 'sagemaker/DEMO-xgboost-dm'
my_region = boto3.session.Session().region_name # set the region of the instance

# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
xgboost_container = sagemaker.image_uris.retrieve("xgboost", my_region, "latest")

print("Success - the MySageMakerInstance is in the " + my_region + " region. You will use the " + xgboost_container + " container for your SageMaker endpoint.")

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


Success - the MySageMakerInstance is in the ap-south-1 region. You will use the 991648021394.dkr.ecr.ap-south-1.amazonaws.com/xgboost:latest container for your SageMaker endpoint.


In [136]:
bucket_name = 'gas-leakage' # <--- CHANGE THIS VARIABLE TO A UNIQUE NAME FOR YOUR BUCKET
s3 = boto3.resource('s3')
try:
    if  my_region == 'us-east-1':
      s3.create_bucket(Bucket=bucket_name)
    else: 
      s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={ 'LocationConstraint': my_region })
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

S3 error:  An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.


In [137]:
# try:
#   urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
#   print('Success: downloaded bank_clean.csv.')
# except Exception as e:
#   print('Data load error: ',e)

# try:
#   model_data = pd.read_csv('./bank_clean.csv',index_col=0)
#   print('Success: Data loaded into dataframe.')
# except Exception as e:
#     print('Data load error: ',e)

In [138]:
# train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
# print(train_data.shape, test_data.shape)

In [139]:
# model_data

In [140]:
# train_data

In [141]:
# pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('train.csv', index=False, header=False)
# boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
# s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')
# tr=pd.read_csv('train.csv')
# tr

In [142]:
try:
  model_data = pd.read_csv('dataset.csv',delimiter=',')
  print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: Data loaded into dataframe.


In [143]:
model_data

,Alcohol,CH4,CO,H2,LPG,Propane,Smoke,Temp,LPG_Leakage
0,0.002335,3.047929,3.151109,18.369012,0.020111,0.007674,0.021269,31,1
1,0.003798,2.084298,2.771445,26.249791,0.011426,0.008733,0.023268,37,1
2,0.004403,4.325731,4.675765,23.568104,0.060044,0.008404,0.017806,36,1
3,0.004409,2.384824,3.655571,19.922499,0.088107,0.008073,0.027534,26,0
4,0.002766,5.026507,3.751914,24.523898,0.074812,0.009797,0.027073,35,1
...,...,...,...,...,...,...,...,...,...
995,0.003156,1.991414,4.859238,13.278470,0.009032,0.007947,0.026099,25,0
996,0.003676,2.566385,0.886917,16.425782,0.077939,0.009784,0.014105,38,0
997,0.003999,5.099325,1.117241,1.042428,0.038019,0.009259,0.011302,35,0
998,0.004100,2.401628,5.892976,19.882149,0.084168,0.006032,0.016403,31,1


In [144]:
# train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
# print(train_data.shape, test_data.shape)

# model_data = model_data.reset_index()
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)


(700, 9) (300, 9)


In [145]:
# df = model_data.sort_values(by=['index'])

# # Reset the index
# df = df.reset_index(drop=True)

# # Save the sorted dataset
# df.to_csv('sorted_dataset.csv', index=False)

In [146]:
# try:
#   model_data = pd.read_csv('sorted_dataset.csv',index_col=0,delimiter=',')
#   print('Success: Data loaded into dataframe.')
# except Exception as e:
#     print('Data load error: ',e)

In [147]:
model_data

,Alcohol,CH4,CO,H2,LPG,Propane,Smoke,Temp,LPG_Leakage
0,0.002335,3.047929,3.151109,18.369012,0.020111,0.007674,0.021269,31,1
1,0.003798,2.084298,2.771445,26.249791,0.011426,0.008733,0.023268,37,1
2,0.004403,4.325731,4.675765,23.568104,0.060044,0.008404,0.017806,36,1
3,0.004409,2.384824,3.655571,19.922499,0.088107,0.008073,0.027534,26,0
4,0.002766,5.026507,3.751914,24.523898,0.074812,0.009797,0.027073,35,1
...,...,...,...,...,...,...,...,...,...
995,0.003156,1.991414,4.859238,13.278470,0.009032,0.007947,0.026099,25,0
996,0.003676,2.566385,0.886917,16.425782,0.077939,0.009784,0.014105,38,0
997,0.003999,5.099325,1.117241,1.042428,0.038019,0.009259,0.011302,35,0
998,0.004100,2.401628,5.892976,19.882149,0.084168,0.006032,0.016403,31,1


In [148]:
pd.concat([train_data['LPG_Leakage'], train_data.drop(['LPG_Leakage'], axis=1)], axis=1).to_csv('train.csv', index=False, header=False)
tr=pd.read_csv('train.csv')
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')

s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')
tr

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


,0,0.003439164,5.395465687,2.064007885,6.300035369,0.06503306,0.006277441,0.011845864,29
0,0,0.004283,5.217901,0.649860,6.415555,0.082717,0.006177,0.011194,33
1,0,0.002628,0.528394,4.086357,25.433765,0.008721,0.008041,0.014468,36
2,0,0.003785,2.366048,5.625322,24.995547,0.035644,0.008532,0.024563,29
3,1,0.002701,1.629464,3.666159,14.887042,0.017448,0.008211,0.012945,34
4,0,0.003499,2.122050,0.607898,2.432415,0.040989,0.009553,0.026905,28
...,...,...,...,...,...,...,...,...,...
694,0,0.004828,3.549830,1.770449,10.454143,0.017494,0.007957,0.010216,37
695,1,0.003171,3.213945,5.032617,13.743637,0.091557,0.009462,0.016168,40
696,1,0.002945,2.290055,4.985377,6.517716,0.049561,0.008450,0.014086,36
697,0,0.002912,1.213213,1.534482,14.729692,0.066298,0.009058,0.013174,37


In [149]:
# try:
#   urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
#   print('Success: downloaded bank_clean.csv.')
# except Exception as e:
#   print('Data load error: ',e)

In [150]:
sess = sagemaker.Session()
xgb = sagemaker.estimator.Estimator(xgboost_container,role, instance_count=1, instance_type='ml.m4.xlarge',output_path='s3://{}/{}/output'.format(bucket_name, prefix),sagemaker_session=sess)
xgb.set_hyperparameters(max_depth=5,eta=0.2,gamma=4,min_child_weight=6,subsample=0.8,silent=0,objective='binary:logistic',num_round=100)

In [151]:
s3_input_train

In [152]:
xgb.fit({'train': s3_input_train})

INFO:sagemaker:Creating training-job with name: xgboost-2023-04-10-13-54-15-934


2023-04-10 13:54:16 Starting - Starting the training job...
2023-04-10 13:54:40 Starting - Preparing the instances for training......
2023-04-10 13:55:32 Downloading - Downloading input data...
2023-04-10 13:56:03 Training - Downloading the training image...
2023-04-10 13:56:33 Training - Training image download completed. Training in progress..Arguments: train
[2023-04-10:13:56:47:INFO] Running standalone xgboost training.
[2023-04-10:13:56:47:INFO] Path /opt/ml/input/data/validation does not exist!
[2023-04-10:13:56:47:INFO] File size need to be processed in the node: 0.06mb. Available memory size in the node: 8610.08mb
[2023-04-10:13:56:47:INFO] Determined delimiter of CSV input is ','
[13:56:47] S3DistributionType set as FullyReplicated
[13:56:47] 700x8 matrix with 5600 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[13:56:47] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=3
[0]#011train-error:0.

In [153]:
xgb_predictor = xgb.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: xgboost-2023-04-10-13-57-28-718
INFO:sagemaker:Creating endpoint-config with name xgboost-2023-04-10-13-57-28-718
INFO:sagemaker:Creating endpoint with name xgboost-2023-04-10-13-57-28-718


------!

In [154]:
from sagemaker.serializers import CSVSerializer

test_data_array = test_data.drop(['LPG_Leakage'], axis=1).values #load the data into an array
xgb_predictor.serializer = CSVSerializer() # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

(300,)


In [155]:
# cm = pd.crosstab(index=test_data['LPG_Leakage'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
# tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
# print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
# print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
# print("Observed")
# print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
# print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


from sklearn.metrics import confusion_matrix

cm = confusion_matrix(test_data['LPG_Leakage'], np.round(predictions_array))
tn = cm[0, 0]
fn = cm[1, 0]
tp = cm[1, 1]
fp = cm[0, 1]
p = (tp + tn) / (tp + tn + fp + fn) * 100

print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No LPG Leakage", "LPG Leakage"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No LPG Leakage", tn / (tn + fn) * 100, tn, fp / (tp + fp) * 100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("LPG Leakage", fn / (tn + fn) * 100, fn, tp / (tp + fp) * 100, tp))



Overall Classification Rate: 98.3%

Predicted      No LPG Leakage LPG Leakage
Observed
No LPG Leakage 99% (180)     3% (4)
LPG Leakage     1% (1)     97% (115) 



In [ ]:
# INFO:sagemaker:Creating model with name: xgboost-2023-04-10-13-57-28-718
# INFO:sagemaker:Creating endpoint-config with name xgboost-2023-04-10-13-57-28-718
# INFO:sagemaker:Creating endpoint with name xgboost-2023-04-10-13-57-28-718

In [161]:
xgb_predictor.delete_endpoint(delete_endpoint_config=True)

INFO:sagemaker:Deleting endpoint configuration with name: xgboost-2023-04-10-13-57-28-718
INFO:sagemaker:Deleting endpoint with name: xgboost-2023-04-10-13-57-28-718
